In [1]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
import tensorflow.keras.layers as layers
import tensorflow.keras as keras
from sklearn.model_selection import train_test_split


In [2]:
DATA_PATH = "./data/"
FALL_DATA_PATH = "./data/fall/"
NOTFALL_DATA_PATH = './data/notfall/'

In [3]:
fall_frames = []
for fall_csv in os.listdir(FALL_DATA_PATH):
    fall_csv_path = os.path.join(FALL_DATA_PATH,fall_csv)
    fall_csv_dataframe = pd.read_csv(fall_csv_path,header=None)
    fall_frames.append(fall_csv_dataframe.to_numpy(dtype="float32"))

In [4]:
fall_frames[0][0]

array([ 1.1100000e+00,  1.1100000e+02, -5.9000000e-02, -8.8000000e-01,
       -3.1999999e-01, -7.3338623e+00,  4.5836639e+00, -5.7295799e-01,
        1.5406840e+01, -1.3029065e+01,  5.3267906e+01], dtype=float32)

In [5]:
notfall_frames = []
for notfall_csv in os.listdir(NOTFALL_DATA_PATH):
    notfall_csv_path = os.path.join(NOTFALL_DATA_PATH,notfall_csv)
    notfall_csv_dataframe = pd.read_csv(notfall_csv_path,header=None)
    notfall_frames.append(notfall_csv_dataframe.to_numpy(dtype="float32"))

In [6]:
len(notfall_frames)
notfall_frames[1].shape

(200, 11)

In [7]:
type(fall_frames[0])

numpy.ndarray

In [ ]:
save_path = "saved_model/sensor_model"
save_dir = os.path.dirname(save_path)

In [10]:
fall_frames = np.asarray(fall_frames)
notfall_frames = np.asarray(notfall_frames)

X = np.concatenate((fall_frames,notfall_frames))
Y = np.asarray([1]*len(fall_frames) + [0]*len(notfall_frames))
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.3)
X_test, X_val, Y_test, Y_val = train_test_split(X_test, Y_test, test_size=0.5)

In [1]:
class SensorModel:
    SEQUENCE_SIZE = 200
    INPUT_DIM = 11
    OUTPUT_DIM = 1
    def __init__(self):
        self.model = None
        
    @classmethod
    def load(cls,path):
        sensorModel = cls()
        sensorModel.model = keras.models.load_model(path)
        return sensorModel
        
    def create(self,loss="binary_crossentropy",optimizer="adam",metrics=['accuracy']):
        self.model = keras.Sequential()
        self.model.add(layers.LSTM(64,input_dim=self.INPUT_DIM,input_length=self.SEQUENCE_SIZE))
        self.model.add(layers.Dense(self.OUTPUT_DIM,activation="sigmoid"))
        self.model.compile(loss=loss,optimizer=optimizer,metrics=metrics)
    def train(self,x_train,y_train,x_val,y_val,batch_size=8,epochs=10):
        self.model.fit(x_train,y_train,batch_size=batch_size, epochs=epochs,verbose=1,validation_data=(x_val,y_val))
    def predict(self,x_test):
        return self.model.predict(x_test)
        

In [ ]:
model = SensorModel.load(save_path)
# model.create()
# model.train(X_train,Y_train,X_val,Y_val)
model.predict(X_test)